In [3]:
#import necessary libraries
import requests
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import pprint
import numpy as np
import pandas as pd

In [ ]:
url1 = 'https://tabs.ultimate-guitar.com/tab/misc-soundtrack/a-star-is-born-shallow-chords-2488086'
url2 = 'https://tabs.ultimate-guitar.com/tab/jeff-buckley/hallelujah-chords-198052'
asession = AsyncHTMLSession() #creates Async object

r = await asession.get(url2,timeout=20) #requests website from server and waits for response
#r.html.render(timeout=20)
site = await r.html.arender(timeout=20) #renders in case there is javascript

soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
body = soup.find(class_='js-page js-global-wrapper') #finds main body of website which contains the strum patterns
patterns = body.find_all(class_='_2u0o0') #every strum pattern is within a class called '_2u0o0'


In [ ]:
patterns[0].contents

By cross checking with the website, I've determined that the classes that have '_1yupz' are downstrums, the classes that have '_3N-at' are downstrums, and the classes that have neither of those are beats you don't strum on

In [ ]:
print(len(patterns)) #the song I'm testing with has 3 strum patterns so this list should hold three Tags
print(type(patterns)) #confirming that patterns is a list/ResultSet
print(type(patterns[0].children)) #checking the type for the children of the Tag
print(type(next(patterns[0].children))) 
print(type(next(patterns[0].children)['class']))
print(next(patterns[0].children)['class'])#Returns class string as a list


In [ ]:
strumPatterns = [] #will hold all strum patterns for song with 1=downstrum, 2=upstrum, 0=skip

for pattern in patterns: #iterate through all strum patterns scraped
    currentPattern = [] #will hold current pattern
    count = 0
    for child in pattern.children: #iterate through all strums within the pattern
        className = child['class'] #get class name of the strum to determine if its upstrum, downstrum, or neither
        
        print(className) #print class name for debugging
        
        if '_1yupz' in className: #if it contains this string, it is an downstrum so append a 1
            currentPattern.append(1)
        elif '_3N-at' in className: #if it contains this string, it is an upstrum so append a 2
            currentPattern.append(2)
        else:   #if it contains this string, it is neither so append a 0
            currentPattern.append(0)
        
    strumPatterns.append(currentPattern) #add current pattern to list of all patterns in song
    print("Integer Pattern:",currentPattern)
    print()
           

In [ ]:
print(np.array(strumPatterns)) #should give 3x12 array because the song being tested has three strum patterns with 12 strums each

In [ ]:
#Now ill figure out how to scrape the links for all the top songs
#url for top daily chords songs:
#'https://www.ultimate-guitar.com/explore?type[]=Chords'
asession = AsyncHTMLSession() #creates Async object
URL = 'https://www.ultimate-guitar.com/explore?type[]=Chords'
r = await asession.get(URL,timeout=20) #requests website from server and waits for response

site = await r.html.arender(timeout=20) #renders in case there is javascript

soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html


In [ ]:
songs = soup.find_all(class_='_2KJtL _1mes3 kWOod') #'_1HdUE' is the class that holds the url for each song
print('Number of song links found:', len(songs)) #there should be 50 songs on the page

In [ ]:
print(type(songs[0]['href']),'\n') 
print(songs[0],'\n') #checks that songs[0] is a Tag
print(songs[0]['href']) #checks if href will return the url as string
print('\n',songs[0].text) #checks song name

In [4]:
#combine previous testing to make function that finds and returns strum patterns from given song url
import time



async def getPatterns(url): #function to parse for strum patterns at given url
    start_time = time.time()
    asession = AsyncHTMLSession() #creates Async object
    
    print("Requesting HTML")
    r = await asession.get(url,timeout=30) #requests website from server and waits for response
    #r.html.render(timeout=20)
    
    #print("Get Website --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    print("Rendering JavaScript")
    site = await r.html.arender(timeout=30,wait=2.0) #renders the javascript, NOTE: Currently the speed bottleneck as the javascript is taking too long to render
    #site = r
    print("Render Website --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
    body = soup.find(class_='js-page js-global-wrapper') #finds main body of website which contains the strum patterns
    patterns = body.find_all(class_='_2u0o0') #every strum pattern is within a class called '_2u0o0'
    
    #print("Parse With Soup --- %s seconds ---" % (time.time() - start_time))
    
    strumPatterns = [] #will hold all strum patterns for song with 1=downstrum, 2=upstrum, 0=skip
    
    start_time = time.time()
    
    for pattern in patterns: #iterate through all strum patterns scraped
        currentPattern = [] #will hold current pattern
        count = 0
        for child in pattern.children: #iterate through all strums within the pattern
            className = child['class'] #get class name of the strum to determine if its upstrum, downstrum, or neither
            if '_1yupz' in className: #if it contains this string, it is an downstrum so append a 1
                currentPattern.append(1)
            elif '_3N-at' in className: #if it contains this string, it is an upstrum so append a 2
                currentPattern.append(2)
            else:   #if it contains this string, it is neither so append a 0
                currentPattern.append(0)

        strumPatterns.append(currentPattern) #add current pattern to list of all patterns in song
    
    #print("Parse Strum Patterns --- %s seconds ---" % (time.time() - start_time))    
    return strumPatterns #returns array of strum patterns, with each row being a strum pattern


In [5]:
#test function
test = await getPatterns('https://tabs.ultimate-guitar.com/tab/green-day/boulevard-of-broken-dreams-chords-146744')
#test = await getPatterns('https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589')
test

Requesting HTML
Rendering JavaScript
Render Website --- 4.56159782409668 seconds ---


[[1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2], [1, 1, 1, 1, 1, 1, 1, 1]]

In [ ]:
for song in songs:
    print(await getPatterns(song['href']),'\n')

In [5]:
#input strum pattern as list with 0=skip,1=downstrum,2=upstrum, numSongs is the number of the top songs you want to check
async def searchStrumPattern(pattern,numSongs): 
    asession = AsyncHTMLSession() #creates Async object
    page = 1 #starts on page 1 of songs website
    URL = 'https://www.ultimate-guitar.com/explore?type[]=Chords&page='
    count = 0 #will hold # of songs checked
    while count < numSongs and page < 10: #each page holds 50 songs so sets limits on number of pages checked
        r = await asession.get(URL+str(page),timeout=30) #requests website from server and waits for response

        site = await r.html.arender(timeout=30) #renders in case there is javascript

        soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
        songs = soup.find_all(class_='_2KJtL _1mes3 kWOod') #'_1HdUE' is the class that holds the url for each song
        
        for song in songs:
            patterns = await getPatterns(song['href'])
            for row in patterns:
                if pattern == row:
                    print(song['href'],'\n',song.text,'\n')
            count += 1
            print(count)
        page += 1


In [ ]:
await searchStrumPattern([1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2],200) #should be strum pattern of Boulevard of Broken Dreams which is on
#[1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2]



In [6]:
async def scrapePatterns(numSongs): 
    asession = AsyncHTMLSession() #creates Async object
    page = 1 #starts on page 1 of songs website
    URL = 'https://www.ultimate-guitar.com/explore?type[]=Chords&page='
    count = 0 #will hold # of songs checked
    data = []
    while count < numSongs and page < 10: #each page holds 50 songs so sets limits on number of pages checked
        r = await asession.get(URL+str(page),timeout=30) #requests website from server and waits for response
        
        site = await r.html.arender(timeout=30) #renders in case there is javascript

        soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
        songs = soup.find_all(class_='_2KJtL _1mes3 kWOod') #'_1HdUE' is the class that holds the url for each song
        print("check")
        for song in songs:
            try:
                print(song['href'])
                patterns = await getPatterns(song['href'])
                data.append((song.text,song['href'],patterns))
            except:
                print('Rendering Timeout after',count)
                
            count += 1
            if count > numSongs:
                break
            print(count,'\n')
        page += 1
    return data

In [ ]:
x = await scrapePatterns(400)

check
https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589
Requesting HTML
Rendering JavaScript
Render Website --- 4.232844114303589 seconds ---

1 

https://tabs.ultimate-guitar.com/tab/elvis-presley/cant-help-falling-in-love-chords-1086983
Requesting HTML
Rendering JavaScript
Render Website --- 4.597390174865723 seconds ---

2 

https://tabs.ultimate-guitar.com/tab/jeff-buckley/hallelujah-chords-198052
Requesting HTML
Rendering JavaScript
Render Website --- 4.231257915496826 seconds ---

3 

https://tabs.ultimate-guitar.com/tab/sia/snowman-chords-2233623
Requesting HTML
Rendering JavaScript
Render Website --- 4.184957265853882 seconds ---

4 

https://tabs.ultimate-guitar.com/tab/passenger/let-her-go-chords-1137467
Requesting HTML
Rendering JavaScript
Render Website --- 4.532984972000122 seconds ---

5 

https://tabs.ultimate-guitar.com/tab/eagles/hotel-california-chords-46190
Requesting HTML
Rendering JavaScript
Render Website --- 4.6913840770721436 seconds ---

6 

Rendering JavaScript
Render Website --- 6.06594705581665 seconds ---

49 

https://tabs.ultimate-guitar.com/tab/ed-sheeran/i-see-fire-chords-1430599
Requesting HTML
Rendering JavaScript
Render Website --- 5.662531852722168 seconds ---

50 

check
https://tabs.ultimate-guitar.com/tab/john-legend/all-of-me-chords-1248578
Requesting HTML
Rendering JavaScript
Render Website --- 5.434646844863892 seconds ---

51 

https://tabs.ultimate-guitar.com/tab/ricky-montgomery/line-without-a-hook-chords-1830299
Requesting HTML
Rendering JavaScript
Render Website --- 5.615716934204102 seconds ---

52 

https://tabs.ultimate-guitar.com/tab/david-bowie/space-oddity-chords-105869
Requesting HTML
Rendering JavaScript
Render Website --- 5.3891870975494385 seconds ---

53 

https://tabs.ultimate-guitar.com/tab/israel-kamakawiwoole/over-the-rainbow-chords-2135261
Requesting HTML
Rendering JavaScript
Render Website --- 5.749680757522583 seconds ---

54 

https://tabs.ultimate-guitar.com/tab/the-beatles/hey-ju

Rendering JavaScript
Render Website --- 5.406666994094849 seconds ---

97 

https://tabs.ultimate-guitar.com/tab/taylor-swift/right-where-you-left-me-chords-3474326
Requesting HTML
Rendering JavaScript
Render Website --- 4.798654317855835 seconds ---

98 

https://tabs.ultimate-guitar.com/tab/rihanna/stay-chords-1195964
Requesting HTML
Rendering JavaScript
Render Website --- 7.217517137527466 seconds ---

99 

https://tabs.ultimate-guitar.com/tab/taylor-swift/no-body-no-crime-chords-3461903
Requesting HTML
Rendering JavaScript
Render Website --- 5.484174013137817 seconds ---

100 

check
https://tabs.ultimate-guitar.com/tab/chris-isaak/wicked-game-chords-11066
Requesting HTML
Rendering JavaScript
Render Website --- 4.9788758754730225 seconds ---

101 

https://tabs.ultimate-guitar.com/tab/death-cab-for-cutie/i-will-follow-you-into-the-dark-chords-335735
Requesting HTML
Rendering JavaScript
Render Website --- 5.45920991897583 seconds ---

102 

https://tabs.ultimate-guitar.com/tab/bob-m

Rendering JavaScript
Render Website --- 8.07541298866272 seconds ---

145 

https://tabs.ultimate-guitar.com/tab/a-great-big-world/say-something-chords-1443639
Requesting HTML
Rendering JavaScript
Render Website --- 5.79737401008606 seconds ---

146 

https://tabs.ultimate-guitar.com/tab/randy-newman/youve-got-a-friend-in-me-chords-107274
Requesting HTML
Rendering JavaScript
Render Website --- 5.654165029525757 seconds ---

147 

https://tabs.ultimate-guitar.com/tab/the-beatles/while-my-guitar-gently-weeps-chords-17446
Requesting HTML
Rendering JavaScript
Render Website --- 5.671404123306274 seconds ---

148 

https://tabs.ultimate-guitar.com/tab/train/hey-soul-sister-chords-884388
Requesting HTML
Rendering JavaScript
Rendering Timeout after 148
149 

https://tabs.ultimate-guitar.com/tab/bob-dylan/blowin-in-the-wind-chords-14835
Requesting HTML
Rendering JavaScript
Render Website --- 5.748700857162476 seconds ---

150 

check
https://tabs.ultimate-guitar.com/tab/misc-soundtrack/grease-

Rendering JavaScript
Render Website --- 5.298161268234253 seconds ---

193 

https://tabs.ultimate-guitar.com/tab/corinne-bailey-rae/put-your-records-on-chords-1079862
Requesting HTML
Rendering JavaScript
Render Website --- 5.434197902679443 seconds ---

194 

https://tabs.ultimate-guitar.com/tab/taylor-swift/exile-chords-3261398
Requesting HTML
Rendering JavaScript
Render Website --- 5.634316921234131 seconds ---

195 

https://tabs.ultimate-guitar.com/tab/tom-odell/another-love-chords-1198980
Requesting HTML
Rendering JavaScript
Render Website --- 5.468280792236328 seconds ---

196 

https://tabs.ultimate-guitar.com/tab/radiohead/creep-chords-51053
Requesting HTML
Rendering JavaScript
Render Website --- 5.376749753952026 seconds ---

197 

https://tabs.ultimate-guitar.com/tab/fools-garden/lemon-tree-chords-607747
Requesting HTML
Rendering JavaScript
Render Website --- 5.656362295150757 seconds ---

198 

https://tabs.ultimate-guitar.com/tab/a-ha/take-on-me-chords-1842621
Requesting H

RuntimeError: This event loop is already running

[E:pyppeteer.connection] connection unexpectedly closed
Task exception was never retrieved
future: <Task finished name='Task-48426' coro=<Connection._async_send() done, defined at /Users/mahuja/opt/anaconda3/lib/python3.8/site-packages/pyppeteer/connection.py:69> exception=InvalidStateError('invalid state')>
Traceback (most recent call last):
  File "/Users/mahuja/opt/anaconda3/lib/python3.8/site-packages/websockets/protocol.py", line 827, in transfer_data
    message = await self.read_message()
  File "/Users/mahuja/opt/anaconda3/lib/python3.8/site-packages/websockets/protocol.py", line 895, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
  File "/Users/mahuja/opt/anaconda3/lib/python3.8/site-packages/websockets/protocol.py", line 971, in read_data_frame
    frame = await self.read_frame(max_size)
  File "/Users/mahuja/opt/anaconda3/lib/python3.8/site-packages/websockets/protocol.py", line 1047, in read_frame
    frame = await Frame.read(
  File "/Users/

In [8]:
x

[('Perfect',
  'https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589',
  [[1, 1, 1, 1, 1, 1], [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1]]),
 ('Cant Help Falling In Love',
  'https://tabs.ultimate-guitar.com/tab/elvis-presley/cant-help-falling-in-love-chords-1086983',
  [[1, 1, 1, 1, 1, 1]]),
 ('Hallelujah (ver\xa02)',
  'https://tabs.ultimate-guitar.com/tab/jeff-buckley/hallelujah-chords-198052',
  [[1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2],
   [1, 0, 0, 1, 0, 2, 1, 0, 0, 1, 0, 2],
   [1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2]]),
 ('Snowman (ver\xa03)',
  'https://tabs.ultimate-guitar.com/tab/sia/snowman-chords-2233623',
  [[1, 0, 2, 1, 1, 1, 1, 0, 2, 1, 1, 1]]),
 ('Hotel California',
  'https://tabs.ultimate-guitar.com/tab/eagles/hotel-california-chords-46190',
  [[1,
    0,
    0,
    2,
    0,
    2,
    0,
    2,
    1,
    0,
    1,
    2,
    1,
    0,
    1,
    2,
    1,
    0,
    0,
    2,
    0,
    2,
    0,
    2,
    1,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
 

In [11]:
dataframe = pd.DataFrame(x,columns=['Song','URL','StrumPattern'])
dataframe

,Song,URL,StrumPattern
0,Perfect,https://tabs.ultimate-guitar.com/tab/ed-sheera...,"[[1, 1, 1, 1, 1, 1], [1, 0, 0, 1, 0, 0, 1, 0, ..."
1,Cant Help Falling In Love,https://tabs.ultimate-guitar.com/tab/elvis-pre...,"[[1, 1, 1, 1, 1, 1]]"
2,Hallelujah (ver 2),https://tabs.ultimate-guitar.com/tab/jeff-buck...,"[[1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2], [1, 0, ..."
3,Snowman (ver 3),https://tabs.ultimate-guitar.com/tab/sia/snowm...,"[[1, 0, 2, 1, 1, 1, 1, 0, 2, 1, 1, 1]]"
4,Hotel California,https://tabs.ultimate-guitar.com/tab/eagles/ho...,"[[1, 0, 0, 2, 0, 2, 0, 2, 1, 0, 1, 2, 1, 0, 1,..."
...,...,...,...
123,Train Wreck (ver 2),https://tabs.ultimate-guitar.com/tab/james-art...,"[[1, 0, 1, 0, 1, 0, 1, 0]]"
124,Three Little Birds,https://tabs.ultimate-guitar.com/tab/bob-marle...,"[[0, 1, 0, 1, 0, 1, 0, 1]]"
125,Stand By Me,https://tabs.ultimate-guitar.com/tab/ben-e-kin...,"[[1, 1, 1, 1, 1, 1, 1, 1]]"
126,Dont Think Twice Its All Right (ver 2),https://tabs.ultimate-guitar.com/tab/bob-dylan...,"[[1, 0, 1, 0, 0, 2, 1, 2, 1, 0, 1, 0, 0, 2, 1,..."


In [35]:
query1 = [1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2]
query2 = [1,0,1,2,0,2,1,2]

for num, entries in dataframe.iterrows():
    #print(entries[1],'\n')
    for pattern in entries[2]:
        if pattern == query2:
            print(entries[0],entries[1],'\n')

Your New Boyfriend https://tabs.ultimate-guitar.com/tab/wilbur-soot/your-new-boyfriend-chords-3297218 

Over The Rainbow (ver 3) https://tabs.ultimate-guitar.com/tab/israel-kamakawiwoole/over-the-rainbow-chords-2135261 

Sweet Child O Mine https://tabs.ultimate-guitar.com/tab/guns-n-roses/sweet-child-o-mine-chords-176076 

Radioactive https://tabs.ultimate-guitar.com/tab/imagine-dragons/radioactive-chords-1171909 

Falling https://tabs.ultimate-guitar.com/tab/harry-styles/falling-chords-2926901 

Night Changes https://tabs.ultimate-guitar.com/tab/one-direction/night-changes-chords-1678440 

I Will Follow You Into The Dark (ver 3) https://tabs.ultimate-guitar.com/tab/death-cab-for-cutie/i-will-follow-you-into-the-dark-chords-335735 

Redemption Song (ver 2) https://tabs.ultimate-guitar.com/tab/bob-marley/redemption-song-chords-417640 

Wicked Game https://tabs.ultimate-guitar.com/tab/chris-isaak/wicked-game-chords-11066 

